In [8]:

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Binning APPLICATION_TYPE and CLASSIFICATION with a specified cutoff
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 100].index.tolist()

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
y = application_df_encoded['IS_SUCCESSFUL'].values
X = application_df_encoded.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

"""## Hyperparameter Tuning"""

# Define the model-building function
def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            units=hp.Int('units_layer1', min_value=32, max_value=512, step=32),
            activation='relu',
            input_dim=X_train_scaled.shape[1]
        ),
        tf.keras.layers.Dense(
            units=hp.Int('units_layer2', min_value=16, max_value=256, step=32),
            activation='relu'
        ),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Initialize the tuner and perform hyperparameter tuning
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best layer 1 units: {best_hps.get('units_layer1')}
Best layer 2 units: {best_hps.get('units_layer2')}
Best learning rate: {best_hps.get('learning_rate')}
""")

# ModelCheckpoint to save the best model during training
checkpoint_filepath = 'AlphabetSoupCharity_Optimized.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

# Train the model
model = tuner.hypermodel.build(best_hps)
model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])



# Load the saved model and recompile it
model = tf.keras.models.load_model('AlphabetSoupCharity_Optimized.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")




Reloading Tuner from my_dir/intro_to_kt/tuner0.json

Best layer 1 units: 512
Best layer 2 units: 16
Best learning rate: 0.001

Epoch 1/50
850/858 [============================>.] - ETA: 0s - loss: 0.5637 - accuracy: 0.7215
Epoch 1: val_accuracy improved from -inf to 0.72420, saving model to AlphabetSoupCharity_Optimized.h5
858/858 [==============================] - 3s 2ms/step - loss: 0.5636 - accuracy: 0.7217 - val_loss: 0.5607 - val_accuracy: 0.7242
Epoch 2/50
 83/858 [=>............................] - ETA: 1s - loss: 0.5678 - accuracy: 0.7157

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


856/858 [============================>.] - ETA: 0s - loss: 0.5528 - accuracy: 0.7289
Epoch 2: val_accuracy did not improve from 0.72420
858/858 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7290 - val_loss: 0.5597 - val_accuracy: 0.7233
Epoch 3/50
836/858 [============================>.] - ETA: 0s - loss: 0.5498 - accuracy: 0.7297
Epoch 3: val_accuracy improved from 0.72420 to 0.72580, saving model to AlphabetSoupCharity_Optimized.h5
858/858 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7295 - val_loss: 0.5569 - val_accuracy: 0.7258
Epoch 4/50
837/858 [============================>.] - ETA: 0s - loss: 0.5491 - accuracy: 0.7316
Epoch 4: val_accuracy did not improve from 0.72580
858/858 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7320 - val_loss: 0.5552 - val_accuracy: 0.7235
Epoch 5/50
837/858 [============================>.] - ETA: 0s - loss: 0.5478 - accuracy: 0.7317
Epoch 5: val_accuracy impro